# Check AirQuality Stations

Filtre air quality sites in order to obtain those with the main requisites needed to our study.                                      

- Have data from any of the following pollutants:                                                                                                                                                                                       
    NO2, NO, O3, PM10, PM2.5                                                                                                                                                                                                          
- Don't miss any year of data  